In [21]:
import json

f = open("master.json", encoding="utf8")

publications = json.loads(f.read())


def create_author_dict(publications):
    author_dict = {}
    for p in publications:
        for a in p["authors"]:
            fullname = a["fname"].lower() + a["lname"].lower()
            if fullname not in author_dict.keys():
                author_dict.update({fullname:[]})
                author_dict[fullname].append(p["paper"]["title"].lower())
            else:
                author_dict[fullname].append(p["paper"]["title"].lower())
    return(author_dict)
        
def create_keyword_dict(publications):
    keyword_dict = {}
    for p in publications:
        for k in p["paper"]["keywords"]:
            if k.lower() not in keyword_dict.keys():
                keyword_dict.update({k.lower():[]})
                keyword_dict[k.lower()].append(p["paper"]["title"].lower())
            else:
                keyword_dict[k.lower()].append(p["paper"]["title"].lower())
    return(keyword_dict)
    
def create_journal_dict(publications):
    journal_dict = {}
    for p in publications: 
        if p["source"]["journal"].lower() not in journal_dict.keys():
            journal_dict.update({p["source"]["journal"].lower():[]})
            journal_dict[p["source"]["journal"].lower()].append(p["paper"]["title"].lower())
        else:
            journal_dict[p["source"]["journal"].lower()].append(p["paper"]["title"].lower())
    return(journal_dict)
        
def create_title_list(publications):
    title_list = []
    for p in publications:
        title_list.append(p["paper"]["title"].lower())
    return(title_list)

def create_title_dict(publications):
    title_dict = {}
    for p in publications:
        if p["paper"]["title"].lower() not in title_dict.keys():
            title_dict[p["paper"]["title"].lower()]={"year":p["year"]}
    return title_dict
                
                
def menu():
    field_list = []
    author = input("First or last name of author:").lower().split(",")
    journal = input("Name of journal:").lower().split(",")
    title = input("Title of article:").lower().split(",")
    keyword = input("Keyword").lower().split(",")
    field_list = [author, journal, title, keyword] #note order
    #any_or_all = input("Do you want to include all or any of the fields?")
    return(field_list, any_or_all)


def flatten(t):
    return [item for sublist in t for item in sublist]

def search(d, i):
    res = [val for key,val in d.items() if i in key]
    res = set(flatten(res))
    return(res)

def combine_search(field_list, any_or_all):
    if any_or_all == "all":
        if field_list[0] != ['']:
            for i in field_list[0]:
                res1 = search(author_dict,i)  
        else:
            res1 = set(title_list)
        if field_list[1] != ['']:
            for i in field_list[1]:
                res2 = search(journal_dict,i)
        else:
            res2 = set(title_list)
        if field_list[2] != ['']:
            for i in field_list[2]:
                res3 = set([t for t in title_list if i in t])
        else:
            res3 = set(title_list)
        if field_list[3] != ['']:
            for i in field_list[3]:
                res4 = search(keyword_dict,i)
        else:
            res4 = set(title_list)

        all_sets = [res1, res2, res3, res4]
        return(all_sets[0].intersection(*all_sets))

    if any_or_all == "or":
        if field_list[0] != ['']:
            for i in field_list[0]:
                res1 = search(author_dict,i)  
        else:
            res1 = set()
        if field_list[1] != ['']:
            for i in field_list[1]:
                res2 = search(journal_dict,i)
        else:
            res2 = set()
        if field_list[2] != ['']:
            for i in field_list[2]:
                res3 = [t for t in title_list if field_list[2] in t]
                res3 = set(res3)
        else:
            res3 = set()
        if field_list[3] != ['']:
            for i in field_list[3]:
                res4 = search(keyword_dict,i)
        else:
            res4 = set()
        all_sets = [res1, res2, res3, res4]
        return(all_sets[0].union(*all_sets))





        
